In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import multiprocessing as mp
from itertools import product

In [2]:
# for G, current_df in product(G_list, df_list):
def triangle_counter(args):
    G, current_df = args
    desired_trigs = 0
    for i in range( len(current_df) ):
        edge = current_df.iloc[i]['from'], current_df.iloc[i]['to']
    #     print(edge)
    #     edge = pos_df.iloc[i]['from'], pos_df.iloc[i]['to']

        if (edge[0] in G.nodes()) and (edge[1] in G.nodes()):
            source_neighbors = list( G.successors(edge[0]) )
            target_neighbors = list( G.successors(edge[1]) )
            desired_trigs += len( (set(source_neighbors) & set(target_neighbors))   - {edge[0], edge[1]} )

            source_neighbors = list( G.predecessors(edge[0]) )
            target_neighbors = list( G.predecessors(edge[1]) )
            desired_trigs += len( (set(source_neighbors) & set(target_neighbors))  - {edge[0], edge[1]} )

            source_neighbors = list( G.predecessors(edge[0]) )
            target_neighbors = list( G.successors(edge[1]) )
            desired_trigs += len( (set(source_neighbors) & set(target_neighbors)) - {edge[0], edge[1]} ) 

            source_neighbors = list( G.successors(edge[0]) )
            target_neighbors = list( G.predecessors(edge[1]) )
            desired_trigs += len( (set(source_neighbors) & set(target_neighbors)) - {edge[0], edge[1]} ) 

    #         source_neighbors = list( G.successors(edge[0]) )
    #         target_neighbors = list( G.successors(edge[1]) )
    #         pos_pos_neg_trigs += len( (set(source_neighbors) & set(target_neighbors)) - {edge[0], edge[1]} ) 


            if (i % 50000) == 0:
                print(i)
#     setting_ind += 1
#     desired_trig_array[setting_ind]
    return desired_trigs

In [3]:
def neg_pos_divider(df):
    G = nx.from_pandas_edgelist(df, source = 'from', target = 'to', edge_attr = 'sign', create_using=nx.DiGraph)

    G.number_of_edges()

    len(G)

    len(df)

    # desired_sign = 1 # -1
    pos_df = df.loc[ df['sign'] == +1 ]
    neg_df = df.loc[ df['sign'] == -1 ]

    G_pos = nx.from_pandas_edgelist(pos_df, source = 'from', target = 'to', edge_attr = 'sign', create_using=nx.DiGraph)
    G_neg = nx.from_pandas_edgelist(neg_df, source = 'from', target = 'to', edge_attr = 'sign', create_using=nx.DiGraph)
    return pos_df, neg_df, G_pos, G_neg

In [4]:
loc = '../data/preprocessed/'

input_file = 'epinions_edges.csv'
# input_file = 'slashdot_edges.csv'
# input_file = 'wikipedia_edges.csv'

In [5]:
df = pd.read_csv(loc + input_file)

# np.sum( df.duplicated(subset=['from', 'to']) )

In [6]:
df_sh = df.copy(deep=True)

df_sh['sign'] = np.random.permutation( df_sh['sign'] )

In [ ]:
results = []
for df_i, df_o in enumerate( [df, df_sh] ):
    pos_df, neg_df, G_pos, G_neg = neg_pos_divider(df_o)

    G_list = [G_pos, G_neg]
    df_list = [ pos_df, neg_df ]
    jobs = []
    for G, current_df in product(G_list, df_list):
        args = G, current_df
        jobs.append(args)

    with mp.Pool(mp.cpu_count()) as pool:
        p_r = pool.map_async(triangle_counter, jobs)
        res = p_r.get()
    results.append(res)

In [8]:
results_orig = results

In [9]:
results[0][0]/=3
results[0][-1]/=3
results[1][0]/=3
results[1][-1]/=3

In [10]:
triangles_df = pd.DataFrame(columns=['T3', 'T2', 'T1', 'T0'])

In [11]:
triangles_df.loc[0] = np.array(results[0]).astype('int')

In [12]:
triangles_df = triangles_df.T

In [13]:
triangles_df.columns=['$T_i$']

In [14]:
triangles_df['$p(T_i)$'] = triangles_df['$T_i$'] / triangles_df['$T_i$'].sum()

In [15]:
triangles_df['$p_0(T_i)$'] = np.array(results[1]) / np.sum(results[1])

In [16]:
delta = np.sum(results[0]).astype('int')

In [17]:
triangles_df['$s(T_i)$'] = (triangles_df['$T_i$'] - (triangles_df['$p_0(T_i)$'] * delta))  /  np.sqrt( delta * triangles_df['$p_0(T_i)$'] * (1-triangles_df['$p_0(T_i)$']) )

In [18]:
triangles_df

,$T_i$,$p(T_i)$,$p_0(T_i)$,$s(T_i)$
T3,11637474,0.871295,0.618762,1900.229788
T2,927177,0.069418,0.321760,-1974.14737
T1,704144,0.052719,0.056234,-55.767249
T0,87725,0.006568,0.003244,213.660732


In [19]:
triangles_df.to_csv('../data/static-triangle-count/' + 'static-triangle-count-' + input_file)